In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import yaml
from tensorflow.keras.models import model_from_yaml
 
batch_size = 3
 
''' 搭建模型'''
l=tf.keras.layers
model=Sequential()
 
#第一层卷积和池化
model.add(l.Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding='valid',input_shape=(227,227,3),activation='relu'))
model.add(l.BatchNormalization())
model.add(l.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid'))
 
#第二层卷积和池化
model.add(l.Conv2D(256,(5,5),(1,1),padding='same',activation='relu'))
model.add(l.BatchNormalization())
model.add(l.MaxPooling2D((3,3),(2,2),padding='valid'))
 
#第三层卷积
model.add(l.Conv2D(384,(3,3),(1,1),'same',activation='relu'))
 
#第四层卷积
model.add(l.Conv2D(384,(3,3),(1,1),'same',activation='relu'))
 
#第五层卷积和池化
model.add(l.Conv2D(256,(3,3),(1,1),'same',activation='relu'))
model.add(l.MaxPooling2D((3,3),(2,2),'valid'))
 
#全连接层
model.add(l.Flatten())
model.add(l.Dense(4096,activation='relu'))
model.add(l.Dropout(0.5))
 
model.add(l.Dense(4096,activation='relu'))
model.add(l.Dropout(0.5))
 
model.add(l.Dense(1000,activation='relu'))
model.add(l.Dropout(0.5))
 
#输出层
model.add(l.Dense(2,activation='softmax'))
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
 
'''导入图片数据'''
#利用ImageDataGenerator生成一个batch一个batch的数据
 
datagen=ImageDataGenerator(samplewise_center=True,rescale=1.0/255,
                           zoom_range=0.2,
                           width_shift_range=.2, height_shift_range=.2,
                           rotation_range=30,
                           brightness_range=[0.8, 1.2],
                           horizontal_flip=True)   #samplewise_center:使输入数据的每个样本均值为0,rescale:归一化
 
train_generator = datagen.flow_from_directory(
    'D:/image/dataset/image_train',
    classes=['bhg','hg'],
    target_size=(227, 227),
    class_mode='categorical',
    batch_size=batch_size)
 
validation_generator = datagen.flow_from_directory(
    'D:/image/dataset/image_test1',
    classes=['bhg','hg'],
    target_size=(227, 227),
    class_mode='categorical',
    batch_size=batch_size)



'''开始训练'''

checkpoint_filepath = 'best_model4.hdf5'
callback_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, monitor='val_accuracy', mode='max', save_best_only=True)
callback_learningrate = ReduceLROnPlateau(monitor='val_accuracy', mode='max', min_delta=0.03, patience=3, factor=.5, min_lr=0.00001, verbose=1)
callbacks = [callback_checkpoint, callback_learningrate]

model.fit_generator(generator=train_generator,steps_per_epoch=183/3,epochs=30,validation_data=validation_generator,validation_steps=47/3,callbacks=callbacks)
yaml_string = model.to_yaml()  # 保存模型结构到yaml文件
open('./model_architecture4.yaml', 'w').write(yaml_string)
model.save_weights('./AlexNet_model4.h5')  #保存模型参数

Found 183 images belonging to 2 classes.
Found 47 images belonging to 2 classes.
Epoch 1/30
61/61 [==============================] - 24s 398ms/step - loss: 1.5160 - accuracy: 0.6230 - val_loss: 0.6501 - val_accuracy: 0.6809
Epoch 2/30
 9/61 [===>..........................] - ETA: 16s - loss: 0.6083 - accuracy: 0.6667

KeyboardInterrupt: 

In [30]:
'''导入模型'''
model = model_from_yaml(open('./model_architecture4.yaml').read())
model.load_weights('./AlexNet_model4.h5')
 
'''随便输入一张图片测试一下'''
imgs=[]
img=cv2.imread('D:/image/hg. (41).jpg')
img=cv2.resize(img,(227,227))
imgs.append(img)
a=np.array(imgs)
 
result=model.predict(a)
idx=np.argmax(result)
 
if idx==0:
    print('the image is hg\n')
else:
    print('the image is bhg\n')

the image is bhg



In [32]:
import yaml
import numpy as np
import cv2
import urllib.request
from tensorflow.keras.models import model_from_yaml

model = model_from_yaml(open('./model_architecture4.yaml').read())
model.load_weights('./AlexNet_model4.h5')
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

imgs = []
image = url_to_image('http://nimg.ws.126.net/?url=http%3A%2F%2Fdingyue.ws.126.net%2F2021%2F0601%2F536e47d6j00qtzfry002ed000m800gop.jpg&thumbnail=650x2147483647&quality=80&type=jpg')
# plt.imshow(image)
img = cv2.resize(image, (227, 227))
imgs.append(img)
a = np.array(imgs)
result=model.predict(a)
idx = np.argmax(result)

if idx == 0:
    print('the image is yes\n')
else:
    print('the image is no\n')

the image is no

